<img src="paiss.jpg" heigth=45>

# Practical Session : Deep RL

## Agenda

  1. Deep-RL Theory Reminder (15')
  1. DQN (30')
  1. $\epsilon$-DQN (20')
  1. DQN with Replay (20')
  1. Potential problems & solutions (10')
  1. Wrap-up (5')
  
(!) Regular polls/quiz
  
  <img src="logo.png" width="250px" align='center'>

# Deep-RL Theory
   
<img src="rl.png" width="480"/>   


**Goal**: Maximise w.r.t $\pi$ to collect more cumulated discounted reward 
  - $R = \sum \limits_{t=0}^{\infty} \gamma^t r_t$
  - $0<\gamma<1$
  

## Reinforcement Learning for a Markov Decision Process 

Under a fixed policy, $V$ is the value of the state:

<img src="vpi.png" width="480"/>

Under a fixed policy, $Q$ is the value of the tuple (state, action):

<img src="qpi.png" width="640"/>

Remark: after time $t$, actions are chosen by $\pi$ so in expectation we could replace $Q^\pi(s_{t+1}, a_{t+1})$ by $V^\pi(s_{t+1})$

### Bellman‘s optimality principle

The optimal policy $\pi^*$ must respect 

<img src="bellman.png" width=480>

For a fixed policy $\pi$ if the quantity are not equals :
  - either $Q$ is not correctly estimated (policy evaluation issue) 
  - or $\pi$ is not selecting optimal actions, this can be fixed being more greedy wrt $Q$

<img src="evalimprov.png" width=480>


# Deep Q-Learning (DQN)

<img src="dqn-th.png" width=800>

<img src="q-learning.png" width=480>

# Deep-RL Benchmarks

## OpenAI 

  - [Gym](https://gym.openai.com/envs/CartPole-v0/) : set of standard problems / environments

### Simple control problems

<img src="control.png" width=480>

### Famous learn to play Atari

<img src="atari.png" width=480>




In [ ]:
class RLEnvironment:

    def run(self, agent, episodes=100, ...):
        """
        Run the agent.

        Pseudo-code:
        ```
            for i in 1..episodes {

                start new episode

                while episode not finished {

                    ask agent to take action based on current state
                    action is resolved by environment, returning reward and new state

                    <opportunity to feedback agent with (state, reward, new state)>
                    <opportunity to update agent parameters>

                    if new state means agent failed {
                        terminate episode
                    }

                }

                <opportunity to update agent parameters again>

                if last episodes show enough reward {
                    declare task solved
                }

            }

        ```
        """

In [ ]:
class RandomAgent:
    """The world's simplest agent!"""
    def __init__(self, action_space):
        self.action_space = action_space

    def get_action(self, state):
        return self.action_space.sample()

In [ ]:

env.run(RandomAgent(env.action_space), episodes=20, display_policy=True)

### CartPole
  - $x$: position of cart
  - $\theta$: angle of the pole
  - actions: move left/right
  - environment simulates acceleration (cart/pole mass)
<img src="cartpole.gif" width=640>

## 3. DQN in practice
  - [Keras doc](https://keras.io/)

In [ ]:
class DQNAgent(RLDebugger):
    def __init__(self, observation_space, action_space):
        self.learning_rate = ???
        
    def build_model(self):
        model = Sequential()
        # try different nb neurons    
        # try different layers
        model.add(Dense(10, input_dim=self.state_size, activation='linear'))
        model.add(Dense(self.action_size, activation='linear'))
        # usual activations: 'linear', 'relu', 'tanh', 'sigmoid'
        model.compile(loss=???, optimizer=???(lr=self.learning_rate))
        # usual losses: 'mse', 'logcosh', 'mean_absolute_error'
        model.summary()
        # usual optimizers: Adam, RMSprop, SGD
        return model


In [ ]:

    # get action from model using greedy policy. 
    def get_action(self, state):
        q_value = self.model.predict(state)
        best_action = np.argmax(q_value[0]) #The [0] is because keras outputs a set of predictions of size 1
        return best_action

    # train the target network on the selected action and transition
    def train_model(self, action, state, next_state, reward, done):
        target = self.model.predict(state)
        # We use our internal model in order to estimate the V value of the next state 
        target_val = self.target_model.predict(next_state)
        # Q Learning: target values should respect the Bellman's optimality principle
        if done: #We are on a terminal state
            target[0][action] = reward
        else:
            target[0][action] = reward + self.gamma * (np.amax(target_val))

        # and do the model fit!
        loss = self.model.fit(state, target, verbose=0).history['loss'][0]
        self.record(action, state, target, target_val, loss)        

In [ ]:
agent = DQNAgent(env.observation_space, env.action_space)
env.run(agent, episodes=100, seed=0)

```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense_1 (Dense)              (None, 30)                150       
=================================================================
Total params: 150
Trainable params: 150
Non-trainable params: 0
_________________________________________________________________
Episode 10, Total reward 9.0
Episode 20, Total reward 9.0
Episode 30, Total reward 10.0
Episode 40, Total reward 10.0
Episode 50, Total reward 10.0
Episode 60, Total reward 10.0
Episode 70, Total reward 10.0
Episode 80, Total reward 10.0
Episode 90, Total reward 10.0
Episode 100, Total reward 10.0
Average Total Reward of last 100 episodes: 9.94
```


### Evaluation
- **Objective:** average reward of 200 over 100 episodes
- *First Target:* reach a reward > 50 (at least once)
- We only focus on tuning model parameters for now

___

Your turn now !

```
$ jupyter notebook exercises.ipynb
```

# DQN First Results

### POLL: who reached a reward > 50 ?


### Symptom

![](loss_dqn_1.png)

### POLL: Is it a problem of ...
  - Model capacity (#neurons) ?
  - Loss function ?
  - Activation ?
  - Exploration ?

![](action_dqn_1.png)

*Take away:* Q-learning will converge only if it explores enough actions (hence states)

# DQN with Exploration

Exploration of the states is crucial for performance

  - add an uniform exploration mechanism
  - decrease exploration over time
  
This is our first agent which is going to solve the task. It will typically require to run a few hundred of episodes to collect the data.

In [ ]:
class DQNAgentWithExploration(DQNAgent):
    def __init__(self, observation_space, action_space):
        super(DQNAgentWithExploration, self).__init__(observation_space, action_space)
        # exploration schedule parameters 
        self.t = 0
        self.epsilon = ???
        # TODO store your additional parameters here 

    # decay epsilon
    def update_epsilon(self):
        self.t += 1
        # TODO write the code for your decay  
        self.epsilon = ???

In [ ]:
agent = DQNAgentWithExploration(env.observation_space, env.action_space)
env.run(agent, episodes=500, print_delay=50)

```
Layer (type)                 Output Shape              Param #   
=================================================================
dense_7 (Dense)              (None, 30)                150       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 62        
=================================================================
Total params: 212
Trainable params: 212
Non-trainable params: 0
_________________________________________________________________
Episode 50, Total reward 10.0
Episode 100, Total reward 37.0
Episode 150, Total reward 139.0
Episode 200, Total reward 200.0
```

___

Your turn now !

PS: if your current DQN is really poor (not reaching reward > 10) you can cheat by using:
```
from .spoiler import DQNAgent
```

# DQN/Explore Results


### POLL: who reached a reward > 50 ?

### POLL: who reached a reward > 200 ?

### POLL: which one is DQN ? DQN/Ex ?
| | |
|---|--- |
| ![](state_dqn_2.png) | ![](state_dqne_1.png) |


### POLL: who got the message "task solved" ?


### POLL: who solved in < 200 episodes ?

*hint*: can we optimize gains from past experience ?



## 7. DQN/Ex with Replay

  - So we collect data according to games recorded using a policy which contains an exploration mechanism.

  - The collected data are going to be highly correlated
  
  - This is a good idea to fight it by keeping a common memory along several episodes and to subsample the training batches in this memory.  
 
  - Prioritized Replay - https://arxiv.org/abs/1511.05952 - goes one step further by weighting the sampling

In [ ]:
class DQNAgentWithExplorationAndReplay(DQNAgentWithExploration):
    def __init__(self, observation_space, action_space):
        ...
        # create replay memory using deque
        self.memory = deque(maxlen=???)
        self.batch_size = ???

    def train_model(self, action, state, next_state, reward, done):
        
        # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, done))
        
        if len(self.memory) >= self.train_start:
            ...

___

Your turn now !

# DQN/Ex/Replay Results


### POLL: who got the message "task solved" ?


### POLL: who solved in < 200 episodes ?

### POLL: Have we converged ?

| | |
|--|--|
| ![](dqnee_state_1.png) | ![](dqnee_bellman_1.png)|

### POLL: Have we converged ?

![](dqnee_all_1.png)

In [ ]:
agent.epsilon = 0
agent.memory = deque(maxlen=1)
agent.batch_size = 1
agent.train_start = 1
env.run(agent, episodes=200, print_delay=33)

# Potential Problems/Solution (advanced)

## Double DQN
  - *assumption*: our Q estimates are too optimistic
  - *solution*: defer model update to avoid big jumps in target
  
  <img src="ddqn-th.png" width=480>

  - practically: "freeze" $Q_2$ for several time steps / episodes

## Dueling DQN
  - *assumption*: action is not so important for many states
  - *solution*: separate $Q$ into state and advantage functions $Q(s,a) = V(s) + A(s,a)$
  
  <img src="duel.png" width=140>

___

You can try to implement one of these as an exercise

## Takeaways

### Basics ideas
  - tune the DL model parameters
  - exploration is necessary for Q-learning
  - debug not only with the model loss
  
### Advanced ideas
  - make exploration more efficient (replay)
  - adapt to task specifics (DDQN, Dueling DQN)
  
### Next steps
  - [John Schulman: nuts and bolts of RL research](http://rll.berkeley.edu/deeprlcourse/docs/nuts-and-bolts.pdf)
  - try the Atari gyms... with a GPU :)
  - try Policy Gradient methods